In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import math
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, precision_score, recall_score, f1_score, roc_curve, precision_recall_curve, auc
from sklearn import preprocessing
import csv

# LDAformer

## Data Embedding

In [2]:
class valueEmbedding(nn.Module):
    def __init__(self, d_input, d_model, value_linear, value_sqrt):
        super(valueEmbedding, self).__init__()

        self.value_linear = value_linear
        self.value_sqrt = value_sqrt
        self.d_model = d_model
        self.inputLinear = nn.Linear(d_input, d_model)

    def forward(self, x):
        if self.value_linear:
            x = self.inputLinear(x)
        if self.value_sqrt:
            x = x * math.sqrt(self.d_model)
        return x

class positionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(positionalEmbedding, self).__init__()
        
        pos_emb = torch.zeros(max_len, d_model).float()
        pos_emb.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

        pos_emb[:, 0::2] = torch.sin(position * div_term)
        pos_emb[:, 1::2] = torch.cos(position * div_term)

        pos_emb = pos_emb.unsqueeze(0)
        self.register_buffer('pos_emb', pos_emb)

    def forward(self, x):
        return self.pos_emb[:, :x.size(1)]

class dataEmbedding(nn.Module):
    def __init__(self, d_input, d_model, value_linear, value_sqrt, posi_emb=False, input_dropout=0.05):
        super(dataEmbedding, self).__init__()
        
        self.posi_emb = posi_emb
        self.value_embedding = valueEmbedding(d_input=d_input, d_model=d_model, value_linear=value_linear, value_sqrt=value_sqrt)
        self.positional_embedding = positionalEmbedding(d_model=d_model)
        self.inputDropout = nn.Dropout(input_dropout)

    def forward(self, x):
        if self.posi_emb:
            x = self.value_embedding(x) + self.positional_embedding(x)
        else:
            x = self.value_embedding(x)
        return self.inputDropout(x)

## Scaled Dot-Product Attention
$$\Large{Attention(Q, K, V) = softmax_k(\frac{QK^T}{\sqrt{d_k}}) V} $$

In [3]:
class scaledDotProductAttention(nn.Module):
    def __init__(self):
        super(scaledDotProductAttention, self).__init__()
        
    def forward(self, queries, keys, values):
        # B, L, H, E = queries.shape  # (batch_size, seq_len, n_heads, d_model // n_heads )
        _, S, _, D = values.shape

        scores = torch.einsum("blhe,bshe->bhls", queries, keys) # matmul
        scale = 1./math.sqrt(math.e) # scale
        # no mask
        A = torch.softmax(scale * scores, dim=-1)   # softmax
        V = torch.einsum("bhls, bshd->blhd", A, values) # matmul
        
        return V.contiguous()

## Attention Layer

In [4]:
class attentionLayer(nn.Module):
    def __init__(self, scaled_dot_product_attention, d_model, n_heads):
        super(attentionLayer, self).__init__()

        d_values = d_model // n_heads

        self.scaled_dot_product_attention = scaled_dot_product_attention
        self.query_linear = nn.Linear(d_model, d_values * n_heads)
        self.key_linear = nn.Linear(d_model, d_values * n_heads)
        self.value_linear = nn.Linear(d_model, d_values * n_heads)
        self.out_linear = nn.Linear(d_values * n_heads, d_model)
        self.n_heads = n_heads

    def forward(self, queries, keys, values):
        B, L, _ = queries.shape # (batch_size, seq_len, d_model)
        _, S, _ = keys.shape
        H = self.n_heads

        queries = self.query_linear(queries).view(B, L, H, -1)
        keys = self.key_linear(keys).view(B, S, H, -1)
        values = self.value_linear(values).view(B, S, H, -1)

        out = self.scaled_dot_product_attention(queries, keys, values)
        out = out.transpose(2,1).contiguous()
        out = out.view(B, L, -1)

        return self.out_linear(out)

## Encoder layer

In [5]:
class encoderLayer(nn.Module):
    def __init__(self, attention_layer, d_model, d_ff, add, norm, ff, encoder_dropout=0.05):
        super(encoderLayer, self).__init__()
        
        d_ff = int(d_ff * d_model)
        self.attention_layer = attention_layer

        self.add = add
        self.norm = norm
        self.ff = ff
        
        # point wise feed forward network
        self.feedForward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )
        self.dropout = nn.Dropout(encoder_dropout)

        self.norm1 = nn.LayerNorm(d_model)    # (batch_size, seq_len, d_model)
        self.norm2 = nn.LayerNorm(d_model)


    def forward(self, x):
        new_x = self.attention_layer(x, x, x)
        
        if self.add:
            if self.norm:
                out1 = self.norm1(x + self.dropout(new_x))
                out2 = self.norm2(out1 + self.dropout(self.feedForward(out1)))
            else:
                out1 = x + self.dropout(new_x)
                out2 = out1 +  self.dropout(self.feedForward(out1))
        else:
            if self.norm:
                out1 = self.norm1(self.dropout(new_x))
                out2 = self.norm2(self.dropout(self.feedForward(out1)))
            else:
                out1 = self.dropout(new_x)
                out2 = self.dropout(self.feedForward(out1))
        
        if self.ff:
            return out2
        else:
            return out1

## Encoder

In [6]:
class encoder(nn.Module):
    def __init__(self, encoder_layers):
        super(encoder, self).__init__()
        self.encoder_layers = nn.ModuleList(encoder_layers)

    def forward(self, x):
        # x [B, L, D]
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x)
        return x

## LDAformer

In [7]:
class LDAformer(nn.Module):
    def __init__(self, seq_len, d_input, d_model=10, n_heads=2, e_layers=6, d_ff=2, pos_emb=False, value_linear=True, value_sqrt=False, add=True, norm=True, ff=True, dropout=0.05):
        super(LDAformer, self).__init__()

        # Encoding
        self.data_embedding = dataEmbedding(d_input, d_model, posi_emb=pos_emb, value_linear=value_linear, value_sqrt=value_sqrt, input_dropout=dropout)
        # Encoder
        self.encoder = encoder(
            [
                encoderLayer(
                    attentionLayer(scaledDotProductAttention(), d_model, n_heads),
                    d_model,
                    d_ff,
                    encoder_dropout=dropout,
                    add=add,
                    norm=norm,
                    ff=ff
                ) for l in range(e_layers)
            ]
        )
        self.prediction = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(dropout),
            nn.Linear(seq_len * d_model, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x_embedding = self.data_embedding(x)
        enc_out = self.encoder(x_embedding)
        
        return self.prediction(enc_out).squeeze(-1)

# Data Loader

In [8]:
def get_data(Ai, ij):
    data = []
    for item in ij:
        feature = np.array([Ai[0][item[0]], Ai[0][item[1]]])
        for dim in range(1, Ai.shape[0]):
            feature = np.concatenate((feature, np.array([Ai[dim][item[0]], Ai[dim][item[1]]])))
        data.append(feature)
    return np.array(data)

class myDataset(Dataset):
    def __init__(self, dataset, fold, dimension, mode='train') -> None:
        super().__init__()

        self.mode = mode
        Ai = []
        A = np.load('data/ours/' + dataset + '/A_' + str(fold) + '.npy')
        Ai.append(A)
        for i in range(dimension - 1):
            tmp = np.dot(Ai[i], A)
            np.fill_diagonal(tmp, 0)
            tmp = tmp / np.max(tmp)
            Ai.append(copy.copy(tmp))
        Ai = np.array(Ai)
        positive_ij = np.load('data/ours/' + dataset + '/positive_ij.npy')
        negative_ij = np.load('data/ours/' + dataset + '/negative_ij.npy')
        positive5foldsidx = np.load('data/ours/' + dataset + '/positive5foldsidx.npy', allow_pickle=True)
        negative5foldsidx = np.load('data/ours/' + dataset + '/negative5foldsidx.npy', allow_pickle=True)

        if mode == 'test':
            positive_test_ij = positive_ij[positive5foldsidx[fold]['test']]
            positive_train_ij = positive_ij[positive5foldsidx[fold]['train']]
            negative_test_ij = negative_ij[negative5foldsidx[fold]['test']]
            negative_train_ij = negative_ij[negative5foldsidx[fold]['train']]
            positive_test_data = torch.Tensor(get_data(Ai, positive_test_ij))
            positive_train_data = torch.Tensor(get_data(Ai, positive_train_ij))
            negative_test_data = torch.Tensor(get_data(Ai, negative_test_ij))
            negative_train_data = torch.Tensor(get_data(Ai, negative_train_ij))
            self.data = torch.cat((positive_test_data, positive_train_data, negative_test_data, negative_train_data)).transpose(1, 2)
            self.target = torch.Tensor([1] * (len(positive_test_ij) + len(positive_train_ij)) + [0] * (len(negative_test_ij) + len(negative_train_ij)))

        elif mode == 'train':
            positive_train_ij = positive_ij[positive5foldsidx[fold]['train']]
            positive_test_ij = positive_ij[positive5foldsidx[fold]['test']]
            positive_train_data = torch.Tensor(get_data(Ai, positive_train_ij))
            positive_test_data = torch.Tensor(get_data(Ai, positive_test_ij))
            negative_train_ij = negative_ij[negative5foldsidx[fold]['test']][:(len(positive_train_ij) + len(positive_test_ij))] # Attention!
            negative_train_data = torch.Tensor(get_data(Ai, negative_train_ij))
            self.data = torch.cat((positive_train_data, positive_test_data, negative_train_data)).transpose(1, 2)
            self.target = torch.Tensor([1] * (len(positive_train_ij) + len(positive_test_ij)) + [0] * len(negative_train_ij))

        print('Finished reading the {} set of Dataset ({} samples found, each dim = {})'
              .format(mode, len(self.data), self.data.shape[1:]))

    def __getitem__(self, index):
        if self.mode == 'train':
            return self.data[index], self.target[index]
        else:
            return self.data[index], self.target[index]

    def __len__(self):
        return len(self.data)

In [9]:
def prep_dataloader(dataset, fold, mode, dimension, batch_size, n_jobs=0):
    ''' Generates a dataset, then is put into a dataloader. '''
    dataset = myDataset(dataset, fold, dimension, mode=mode)  # Construct dataset
    dataloader = DataLoader(
        dataset, batch_size,
        shuffle=(mode=='train'), drop_last=False,
        num_workers=n_jobs, pin_memory=True)                            # Construct dataloader
    return dataloader

In [10]:
def train(tr_set, model, config, gpu):
    
    criterion = nn.BCELoss()
    n_epochs = config['n_epochs']
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

    loss_record = []     # for recording training loss
    epoch = 0
    while epoch < n_epochs:
        model.train()                           # set model to training mode
        for x, y in tr_set:                     # iterate through the dataloader
            optimizer.zero_grad()               # set gradient to zero
            x, y = x.to(gpu), y.to(gpu)   # move data to device (cpu/cuda)
            pred = model(x)                  # forward pass (compute output)
            bce_loss = criterion(pred, y)  # compute loss
            bce_loss.backward()                 # compute gradient (backpropagation)
            optimizer.step()                    # update model with optimizer
            loss_record.append(bce_loss.detach().cpu().item())
        epoch += 1

        # if bce_loss > 1 or bce_loss < 0.05:
        #     break
        if bce_loss > 1:
            break

        print(epoch, bce_loss.detach().cpu().item())
        
    print('Finished training after {} epochs'.format(epoch))
    return loss_record

In [11]:
def test(tt_set, model, device):
    model.eval()                                # set model to evalutation mode
    preds = []
    labels = []
    for x, y in tt_set:                            # iterate through the dataloader
        x = x.to(device)                     # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            preds.append(pred.detach().cpu())   # collect prediction
        labels.append(y.detach().cpu())
    preds = torch.cat(preds, dim=0).numpy()     # concatenate all predictions and convert to a numpy array
    labels = torch.cat(labels, dim=0).numpy()
    return labels, preds

# Dataset1

In [13]:
gpu = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

config = {
    'n_epochs': 30,
    'batch_size': 32
}

In [14]:
tr_set = prep_dataloader(dataset='dataset1', fold=0, mode='train', dimension=config['dimension'], batch_size=config['batch_size'])
tt_set = prep_dataloader(dataset='dataset1', fold=0, mode='test', dimension=config['dimension'], batch_size=config['batch_size'])

Finished reading the train set of Dataset (5394 samples found, each dim = torch.Size([1147, 6]))
Finished reading the test set of Dataset (98880 samples found, each dim = torch.Size([1147, 6]))


In [23]:
seq_len = tr_set.dataset[0][0].shape[0]
d_input = tr_set.dataset[0][0].shape[1]
while True:
    model = LDAformer(
                seq_len=seq_len, d_input=d_input, d_model=config['d_model'], 
                n_heads=config['n_heads'], d_ff=config['d_ff'], e_layers=config['e_layers'], 
                pos_emb=False, value_sqrt=False
            ).to(gpu)
    model_loss_record = train(tr_set, model, config, gpu)
    if model_loss_record[-1] < 1:
        break
labels, preds = test(tt_set, model, gpu)

AUC = roc_auc_score(labels, preds)
print("AUC =", AUC)
precision, recall, _ = precision_recall_curve(labels, preds)
AUPR = auc(recall, precision)
print('AUPR =', AUPR)

1 0.4368255138397217
2 0.04317177087068558
3 0.2771787643432617
4 0.1632678210735321
5 0.36366787552833557
6 0.09231160581111908
7 0.04903702437877655
8 0.1620827466249466
9 0.15259255468845367
10 0.5268440246582031
11 0.12016847729682922
12 0.4189300537109375
13 0.022369781509041786
14 0.11466704308986664
15 0.011287491768598557
16 0.17592783272266388
17 0.11885125190019608
18 0.20580869913101196
19 0.018127964809536934
20 0.2322070151567459
21 0.07566765695810318
22 0.049617085605859756
23 0.040026355534791946
24 0.1529049128293991
25 0.012376576662063599
26 0.008393464609980583
27 0.03692779317498207
28 0.11788731068372726
29 0.13929125666618347
30 0.4745902419090271
Finished training after 30 epochs
AUC = 0.9961558397800053
AUPR = 0.9030272072528448


In [15]:
# torch.save(model, 'models/LDAformer/case_study.pth')
model = torch.load('models/LDAformer/dataset1_case_study.pth')
labels, preds = test(tt_set, model, gpu)

AUC = roc_auc_score(labels, preds)
print("AUC =", AUC)
precision, recall, _ = precision_recall_curve(labels, preds)
AUPR = auc(recall, precision)
print('AUPR =', AUPR)

AUC = 0.9961558397800053
AUPR = 0.9030272072528448


In [28]:
ds = 'dataset1'
lnc_di = pd.read_csv('data/ours/' + ds + '/lnc_di.csv', index_col=0)
diseases = lnc_di.columns
lncRNAs = lnc_di.index

positive_ij = np.load('data/ours/' + ds + '/positive_ij.npy')
negative_ij = np.load('data/ours/' + ds + '/negative_ij.npy')
positive5foldsidx = np.load('data/ours/' + ds + '/positive5foldsidx.npy', allow_pickle=True)
negative5foldsidx = np.load('data/ours/' + ds + '/negative5foldsidx.npy', allow_pickle=True)
positive_test_ij = positive_ij[positive5foldsidx[0]['test']]
positive_train_ij = positive_ij[positive5foldsidx[0]['train']]
negative_test_ij = negative_ij[negative5foldsidx[0]['test']]
negative_train_ij = negative_ij[negative5foldsidx[0]['train']]

ij = np.concatenate((positive_test_ij, positive_train_ij, negative_test_ij, negative_train_ij))
i = ij[:, 0].T
j = ij[:, 1].T
labels = labels.astype(int)
prediction_results = pd.DataFrame({
        'lncRNA': np.array([lncRNAs[lncRNA] for lncRNA in i]),
        'disease': np.array([diseases[disease - len(lncRNAs)] for disease in j]),
        'pred': preds,
        'label': labels
    })

evidence = pd.read_csv('data/ours/dataset2/union/di_lnc_union.csv', index_col='Unnamed: 0')
evidence_diseases = evidence.index
evidence_lncRNAs = evidence.columns

new_results = pd.DataFrame(columns=['lncRNA', 'disease', 'pred', 'label', 'evidence'])
for idx, row in prediction_results.iterrows():
    lncRNA = row['lncRNA']
    disease = row['disease']
    evd = 0
    if (lncRNA in evidence_lncRNAs) and (disease in evidence_diseases) and (evidence.loc[disease, lncRNA] == 1):
        evd = 1
    new_results = new_results.append({
        'lncRNA': lncRNA,
        'disease': disease,
        'pred': row['pred'],
        'label': row['label'],
        'evidence': evd
    }, ignore_index=True)
new_results.sort_values(by='pred', ascending=False).to_csv('files/case study/dataset1.csv')

In [30]:
for di, case in list(new_results[new_results['evidence'] == 1].groupby(['disease'])):
    if len(case.values) > 10:
        print(di)
        new_results[(new_results['disease'] == di) & (new_results['label'] == 0)].sort_values(by='pred', ascending=False).to_csv('files/case study/' + di + '.csv')

DOID:0050865
DOID:0050866
DOID:10283
DOID:10534
DOID:11054
DOID:1324
DOID:1380
DOID:1612
DOID:162
DOID:1749
DOID:1781
DOID:1793
DOID:1909
DOID:2152
DOID:219
DOID:2394
DOID:2876
DOID:3068
DOID:3070
DOID:3121
DOID:3347
DOID:3498
DOID:3571
DOID:3748
DOID:3907
DOID:3908
DOID:3910
DOID:3969
DOID:4362
DOID:4450
DOID:4467
DOID:47
DOID:5041
DOID:5520
DOID:684
DOID:768
DOID:769
DOID:9119
DOID:9256
DOID:9261
DOID:9538


In [7]:
dataset1_results = pd.read_csv('files/case study/dataset1/dataset1.csv', index_col=0)
dataset1_results

,lncRNA,disease,pred,label,evidence
1369,H19,DOID:162,1.000000,1,1
1652,GAS5,DOID:4,1.000000,1,0
2104,MALAT1,DOID:4,1.000000,1,0
2101,H19,DOID:1612,1.000000,1,1
517,MEG3,DOID:14566,1.000000,1,0
...,...,...,...,...,...
72212,LINC00261,DOID:77,0.000003,0,0
18926,LINC-ROR,DOID:77,0.000003,0,0
64420,LINC00467,DOID:77,0.000003,0,0
51284,FENDRR,DOID:77,0.000002,0,0


In [21]:
len(dataset1_results[(dataset1_results['evidence'] == 1)].values)

1860

In [9]:
len(dataset1_results[((dataset1_results['evidence'] == 1) & (dataset1_results['label'] == 0))].values)

1258

In [19]:
len(dataset1_results[((dataset1_results['evidence'] == 1) & (dataset1_results['label'] == 0) & (dataset1_results['pred'] > 0.5))].values)

280

In [11]:
dataset1_results[((dataset1_results['evidence'] == 1) & (dataset1_results['label'] == 0) & (dataset1_results['pred'] > 0.5))]

,lncRNA,disease,pred,label,evidence
86633,H19,DOID:9119,0.999958,0,1
39858,MALAT1,DOID:8649,0.999939,0,1
91785,EWSAT1,DOID:3347,0.999908,0,1
82481,CDKN2B-AS1,DOID:4362,0.999887,0,1
43584,H19,DOID:114,0.999838,0,1
...,...,...,...,...,...
46852,GAS5,DOID:0050866,0.507397,0,1
69872,MIR100HG,DOID:9256,0.505816,0,1
98688,NEAT1,DOID:3068,0.503372,0,1
67615,PVT1,DOID:0050866,0.502477,0,1


# Dataset2

In [ ]:
gpu = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

config = {
    'n_epochs': 10,
    'batch_size': 32
}

In [17]:
tr_set = prep_dataloader(dataset='dataset2', fold=0, mode='train', dimension=config['dimension'], batch_size=config['batch_size'])
tt_set = prep_dataloader(dataset='dataset2', fold=0, mode='test', dimension=config['dimension'], batch_size=config['batch_size'])

Finished reading the train set of Dataset (7666 samples found, each dim = torch.Size([1276, 6]))
Finished reading the test set of Dataset (210140 samples found, each dim = torch.Size([1276, 6]))


In [14]:
seq_len = tr_set.dataset[0][0].shape[0]
d_input = tr_set.dataset[0][0].shape[1]
while True:
    model = LDAformer(
                seq_len=seq_len, d_input=d_input, d_model=config['d_model'], 
                n_heads=config['n_heads'], d_ff=config['d_ff'], e_layers=config['e_layers'], 
                pos_emb=False, value_sqrt=False
            ).to(gpu)
    model_loss_record = train(tr_set, model, config, gpu)
    if model_loss_record[-1] < 1:
        break
labels, preds = test(tt_set, model, gpu)

AUC = roc_auc_score(labels, preds)
print("AUC =", AUC)
precision, recall, _ = precision_recall_curve(labels, preds)
AUPR = auc(recall, precision)
print('AUPR =', AUPR)

1 0.18491390347480774
2 0.11688053607940674
3 0.452679306268692
4 0.3152638077735901
5 0.14764411747455597
6 0.30353203415870667
7 0.224164679646492
8 0.5397985577583313
9 0.37905147671699524
10 0.28358888626098633
Finished training after 10 epochs
AUC = 0.9574395486089452
AUPR = 0.5151922682331465


In [18]:
# torch.save(model, 'models/LDAformer/dataset2_case_study.pth')
model = torch.load('models/LDAformer/dataset2_case_study.pth')

labels, preds = test(tt_set, model, gpu)

AUC = roc_auc_score(labels, preds)
print("AUC =", AUC)
precision, recall, _ = precision_recall_curve(labels, preds)
AUPR = auc(recall, precision)
print('AUPR =', AUPR)

AUC = 0.9574395486089452
AUPR = 0.5151922682331465


In [19]:
ds = 'dataset2'
lnc_di = pd.read_csv('data/ours/' + ds + '/intersection/di_lnc_intersection.csv', index_col=0)
diseases = lnc_di.index
lncRNAs = lnc_di.columns

positive_ij = np.load('data/ours/' + ds + '/positive_ij.npy')
negative_ij = np.load('data/ours/' + ds + '/negative_ij.npy')
positive5foldsidx = np.load('data/ours/' + ds + '/positive5foldsidx.npy', allow_pickle=True)
negative5foldsidx = np.load('data/ours/' + ds + '/negative5foldsidx.npy', allow_pickle=True)
positive_test_ij = positive_ij[positive5foldsidx[0]['test']]
positive_train_ij = positive_ij[positive5foldsidx[0]['train']]
negative_test_ij = negative_ij[negative5foldsidx[0]['test']]
negative_train_ij = negative_ij[negative5foldsidx[0]['train']]

ij = np.concatenate((positive_test_ij, positive_train_ij, negative_test_ij, negative_train_ij))
i = ij[:, 0].T
j = ij[:, 1].T
labels = labels.astype(int)
prediction_results = pd.DataFrame({
        'lncRNA': np.array([lncRNAs[lncRNA] for lncRNA in i]),
        'disease': np.array([diseases[disease - len(lncRNAs)] for disease in j]),
        'pred': preds,
        'label': labels
    })
# prediction_results.to_csv('files/case study/dataset2/dataset2.csv')
prediction_results[prediction_results['label'] == 0].sort_values(by='pred', ascending=False).to_csv('files/case study/dataset2/dataset2_label0.csv')